# 代码实现ppo

先把本教程中的mask忽略，加入了一些mask写的有点乱

trl代码中的对于ppo的实现
https://github.com/huggingface/trl/blob/main/trl/trainer/ppo_trainer.py

https://mp.weixin.qq.com/s/S72LO26IsZ8AED8sQKIWnQ

讲了PPO  loss max https://zhuanlan.zhihu.com/p/28223597805

https://zhuanlan.zhihu.com/p/677607581

In [ ]:
# ---- 基本超参（教学用简化配置） ----
# 这些超参用于快速搭建一个极小的 GPT-2/ppo 教学环境，便于直观看到张量形状和算法流程。
# 若改为真实训练，请按模型与显存调大 vocab/hidden/layer/head 等参数，并重新生成 tokenizer/权重。
vocab_size = 10   # 词表大小，此处用极小词表便于演示
hidden_size = 128  # GPT2 隐藏维度/embedding 维度
intermediate_size = 256  # FFN 中间层维度，通常是 hidden 的 2-4 倍
num_hidden_layers = 2  # Transformer block 层数（教学例子用 2 层）
num_attention_heads = 4  # 多头注意力的头数
batch_size = 3  # 演示批大小
length_x = 5  # prompt 长度（输入序列长度）
max_new_tokens = 5  # 生成时最多追加的 token 数

## 初始化actor模型

以GPT2为例，初始化模型

In [ ]:
import torch
from transformers import GPT2Config, GPT2LMHeadModel

# 固定随机种子，保证多次运行的参数初始化和示例输出一致，便于教学复现
torch.manual_seed(1)

# ---- 定义 GPT2 配置（演示用小模型） ----
# 下方参数会覆盖前面的超参，确保 config 与后续模型/分词器兼容；这里依然使用小模型以降低计算量。
vocab_size = 10
hidden_size = 128
intermediate_size = 256
num_hidden_layers = 2
num_attention_heads = 4

# 构造 GPT2Config：主要控制词表大小、隐藏维度、层数、注意力头数
# 注意 vocab_size 仍设为官方 GPT-2 词表大小 50257，便于直接复用预训练 tokenizer；
# 如需完全自定义小词表，需要同时训练或加载对应 tokenizer。
config = GPT2Config(
    vocab_size=50257,          # 这里用标准 GPT-2 词表大小，方便加载 tokenizer
    n_embd=hidden_size,        # hidden/embedding 维度
    n_inner=intermediate_size, # FFN 中间层维度
    n_layer=num_hidden_layers, # Transformer block 层数
    n_head=num_attention_heads # 注意力头数
)

# 基于配置实例化一个 GPT-2 语言模型（带 LM 头）；此处不会加载预训练权重，完全随机初始化
model = GPT2LMHeadModel(config)

/home/yxc/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## model generate

主要看下inputs_ids和attention_mask的含义

### inputs_ids

input_ids：它是一个张量（tensor），表示文本被分词后每个词（token）对应的 ID。比如在第一行 [20015, 232, 25465, ...] 中，每个数字都是原文本中一个词被 GPT - 2 分词器转换后的唯一标识。不同模型的词表不同，这些 ID 对应的具体词汇也不一样。这里第一行可能对应一句中文文本分词结果，第二行 [14150, 257, 922, ...] 前半部分对应英文文本，后半部分 50256 一般是填充值 ，表示补齐固定长度。


attention_mask：同样是张量，用于指示哪些位置是有效的词（值为 1），哪些位置是填充的（值为 0） 。比如第二行 [1, 1, 1, 1, 0, 0, 0, 0, 0, 0] 表示前 4 个词是有效输入，后面是填充的，模型在处理时会忽略填充位置。

inputs_ids可以认为是要输入的文本经过tokenizer处理后的结果，而attention_mask则是用于指示哪些位置是有效的词（值为 1），哪些位置是填充的（值为 0） 。

In [ ]:
from transformers import GPT2Tokenizer
import torch

# 初始化 GPT-2 分词器；使用官方权重以复用词表/分词规则
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# GPT-2 原生没有 pad_token，这里把 pad_token 设为 eos 方便 batch padding
# 注意：将 pad 设为 eos 可能会让模型混淆结束符与填充位，后续需配合 attention_mask 使用
# 如需严格区分，可自定义 pad_token 并重新训练/映射词表。
tokenizer.pad_token = tokenizer.eos_token

# 示例输入（中英文各一句）
inputs = ['今天天气不错', 'have a good day']

# 批量分词并右侧填充到同长，返回 PyTorch 张量
# padding=True 触发右填充（默认为 right），truncation=True 避免超长序列
inputs = tokenizer(
    inputs,
    return_tensors='pt',
    padding=True,
    truncation=True
)

print(inputs)  # 打印 input_ids 和 attention_mask，观察填充位置

{'input_ids': tensor([[20015,   232, 25465, 25465, 36365,   242, 38834,   165,   242,   247],
        [14150,   257,   922,  1110, 50256, 50256, 50256, 50256, 50256, 50256]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 0, 0, 0, 0, 0, 0]])}


In [ ]:
# 未显式传入 attention_mask，会触发 warnings 并在 decode 端可能出现异常填充；演示保留，便于对比
output_ids = model.generate(inputs['input_ids'], max_new_tokens=max_new_tokens)
print(output_ids)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


tensor([[20015,   232, 25465, 25465, 36365,   242, 38834,   165,   242,   247,
           247,   247,   247,   247,   247],
        [14150,   257,   922,  1110, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256]])


In [ ]:
# 将生成的 token 序列转回文本，skip_special_tokens=True 以去除 eos/pad 等特殊符号
output_ids = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
print(output_ids)

['今天天气不错�����', 'have a good day']


填充左边和右边会导致input_ids中padding_id的位置不一样，导致attention_mask中padding_id的位置不一样，导致模型在处理时会忽略填充位置。

In [ ]:
# 改为左填充，便于 decoder-only 结构推理时让有效 token 在序列尾部（更符合 GPT 的位置编码习惯）
tokenizer.padding_side = 'left'
inputs = ['今天天气不错', 'have a good day']
inputs = tokenizer(inputs, return_tensors='pt', padding=True, truncation=True)

print(inputs)

# 仍未传入 attention_mask，此处只为对比左右填充对输出的影响
output_ids = model.generate(inputs['input_ids'], max_new_tokens=max_new_tokens)

print(output_ids)

# 解码生成结果，观察因填充方式不同导致的重复等现象
output_ids = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
print(output_ids)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


{'input_ids': tensor([[20015,   232, 25465, 25465, 36365,   242, 38834,   165,   242,   247],
        [50256, 50256, 50256, 50256, 50256, 50256, 14150,   257,   922,  1110]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 1, 1, 1, 1]])}
tensor([[20015,   232, 25465, 25465, 36365,   242, 38834,   165,   242,   247,
           247,   247,   247,   247,   247],
        [50256, 50256, 50256, 50256, 50256, 50256, 14150,   257,   922,  1110,
          1110,  1110,  1110,  1110,  1110]])
['今天天气不错�����', 'have a good day day day day day day']


# 现在开始正式讲rlhf流程

## 初始化reward model

根据之前的定义，奖励模型可以从模型的输出中提取出最后一个token的隐藏状态，然后通过一个线性层计算奖励。

假设batch_size = 2, sequence_length = 4
input_ids = torch.tensor([
    [1, 2, 3, 4],  # 第一个序列
    [5, 6, 7, 8]   # 第二个序列
])

attention_mask = torch.tensor([
    [1, 1, 1, 0],  # 第一个序列有效长度为3
    [1, 1, 1, 1]   # 第二个序列有效长度为4
])

sequence_length = attention_mask.sum(dim=1).long() - 1

结果: tensor([2, 3])

第一个序列：3-1=2（索引从0开始）

第二个序列：4-1=3

batch_indices = torch.arange(batch_size)

结果: tensor([0, 1])

假设hidden_size = 2

last_hidden_state = torch.tensor([
    [[1.0, 1.1], [2.0, 2.1], [3.0, 3.1], [4.0, 4.1]],  # 第一个序列
    [[5.0, 5.1], [6.0, 6.1], [7.0, 7.1], [8.0, 8.1]]   # 第二个序列
])

使用batch_indices和sequence_length提取

result = last_hidden_state[batch_indices, sequence_length]

结果: tensor([[3.0, 3.1],    # 第一个序列的第2个位置（索引从0开始）

[8.0, 8.1]])   # 第二个序列的第3个位置

In [ ]:
class GPTRewardModel(torch.nn.Module):
    def __init__(self, gpt_model, reward_head):
        super(GPTRewardModel, self).__init__()
        self.gpt_model = gpt_model  # 共享 backbone，保持与 actor 一致
        self.reward_head = reward_head  # 将最后一个 token 的隐藏状态映射到标量奖励
        
    def forward(self, input_ids, attention_mask):
        # 获取模型的输出；需打开 output_hidden_states 才能拿到所有层的 hidden_states
        outputs = self.gpt_model(input_ids=input_ids, attention_mask=attention_mask)
        # 通常取最后一个隐藏状态作为输出（hidden_states[-1] 形状: [B, T, H]）
        last_hidden_state = outputs.hidden_states[-1]
        batch_size = input_ids.shape[0]
        # 根据 attention_mask 计算每条序列的有效长度，减 1 得到最后一个有效 token 的索引
        sequence_length = attention_mask.sum(dim=1).long() - 1
        # 使用 batch 维索引选取每条序列的末 token 表征
        batch_indices = torch.arange(batch_size, device=input_ids.device).long()
        last_hidden_state = last_hidden_state[batch_indices, sequence_length]
        print(f"last_hidden_state shape: {last_hidden_state.shape}, sequence_length: {sequence_length.shape}")
        # 计算奖励（线性投影到标量）；输出形状 [B, 1]
        rewards = self.reward_head(last_hidden_state)
        return rewards

# 重新初始化模型并开启 hidden_states 输出，便于 reward/value 头复用
model.config.output_hidden_states = True
rm_model = GPTRewardModel(model, torch.nn.Linear(hidden_size, 1))  # reward_head: H->1

In [9]:
inputs['input_ids']

tensor([[20015,   232, 25465, 25465, 36365,   242, 38834,   165,   242,   247],
        [50256, 50256, 50256, 50256, 50256, 50256, 14150,   257,   922,  1110]])

In [ ]:
# 通过奖励头计算每条序列末 token 的奖励；输出形状 [batch, 1]
reward = rm_model(inputs['input_ids'], inputs['attention_mask'])
print(reward)

last_hidden_state shape: torch.Size([2, 128]), sequence_length: torch.Size([2])
tensor([[-0.1647],
        [-0.2839]], grad_fn=<AddmmBackward0>)


## 简化版ppo
从以上过程可以看出，我们输入给模型的其实是input_ids和attention_mask，所以我们现在为了展示方便，构造一个没有实际意义的输入，输入给模型，然后输出奖励。

In [ ]:
# 构造模拟的 prompt/response 离散 token 序列，仅用于演示，无语义意义
prompt = torch.randint(0, vocab_size, (batch_size, length_x))
response = torch.randint(0, vocab_size, (batch_size, length_x + max_new_tokens))

In [ ]:
# 打印模拟的 prompt 与 response，便于后续对齐 mask 和 logprob
print(prompt)
print(response)

tensor([[5, 0, 0, 1, 0],
        [4, 8, 1, 4, 1],
        [9, 6, 7, 0, 5]])
tensor([[4, 8, 5, 2, 9, 5, 5, 0, 6, 3],
        [0, 3, 0, 4, 8, 2, 6, 4, 9, 3],
        [2, 6, 7, 5, 0, 0, 3, 3, 4, 8]])


我们希望让模型只关注response，所以对prompt对应的mask置为0

In [ ]:
# 为 response 构造 mask：prompt 部分置 0，response 部分置 1，只在生成片段上计算 logprob/value/reward
attention_mask = torch.ones(batch_size, length_x + max_new_tokens)
attention_mask[:, :length_x] = 0
print(attention_mask)


tensor([[0., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 1., 1., 1., 1., 1.]])


In [ ]:
# prompt 的 attention_mask 全 1，若需要将 prompt 也参与 logprob 计算可复用
prompt_attention_mask = torch.ones(batch_size, length_x)
prompt_attention_mask

tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])

创建几个模型


model_ref 和model的配置一样

reward model和value model的配置大体一样

value model的输出是所有token的隐藏状态所得到的value

In [ ]:
# 初始化参考策略模型（ref）；与 actor 使用相同 config，保持权重基线用于 KL 惩罚
model_ref = GPT2LMHeadModel(config)

/opt/anaconda3/envs/llm/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:774: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `output_hidden_states` is. When `return_dict_in_generate` is not `True`, `output_hidden_states` is ignored.
  warnings.warn(


查看区别

In [ ]:
# 对比 reference 与 actor 结构，确认两者一致便于后续 KL 计算
print(model_ref)
print(model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 128)
    (wpe): Embedding(1024, 128)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-1): 2 x GPT2Block(
        (ln_1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=384, nx=128)
          (c_proj): Conv1D(nf=128, nx=128)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=256, nx=128)
          (c_proj): Conv1D(nf=128, nx=256)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=128, out_features=50257, bias=False)
)
GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte)

## 初始化value model

假设我们有以下维度的数据：

last_hidden_state 的形状是 [batch_size, sequence_length, hidden_size]

比如 [5, 10, 128]，表示批次大小为5，序列长度为10，隐藏层维度为128

self.value_head 是一个线性层 Linear(hidden_size, 1)

输入维度是128，输出维度是1

处理过程：

self.value_head(last_hidden_state) 的操作：

输入: [5, 10, 128]

输出: [5, 10, 1] # 线性层将最后一个维度从128转换为1

[:, :, 0] 的操作：

取最后一个维度的第0个元素

结果形状变为: [5, 10]

In [ ]:
class GPTValueModel(torch.nn.Module):
    def __init__(self, gpt_model, value_head):
        super().__init__()
        self.gpt_model = gpt_model  # 共享 backbone
        self.value_head = value_head  # 线性层映射到标量 value
        
    def forward(self, input_ids, attention_mask):
        # 前向推理并取最后一层隐藏状态；形状 [B, T, H]
        outputs = self.gpt_model(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.hidden_states[-1]
        # value_head 输出形状 [B, T, 1]，再去掉最后一维得到 [B, T]
        values = self.value_head(last_hidden_state)[:, :, 0]
        return values
    
# 打开 hidden_states，便于 value_head 读取；重用 actor 权重做 critic（教学示例）
model.config.output_hidden_states = True
vm_model = GPTValueModel(model, torch.nn.Linear(hidden_size, 1))

In [ ]:
# 打印 reward/value 模型，确认头部与 backbone 绑定
print(rm_model)
print(vm_model)

GPTRewardModel(
  (gpt_model): GPT2LMHeadModel(
    (transformer): GPT2Model(
      (wte): Embedding(50257, 128)
      (wpe): Embedding(1024, 128)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0-1): 2 x GPT2Block(
          (ln_1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2SdpaAttention(
            (c_attn): Conv1D(nf=384, nx=128)
            (c_proj): Conv1D(nf=128, nx=128)
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (mlp): GPT2MLP(
            (c_fc): Conv1D(nf=256, nx=128)
            (c_proj): Conv1D(nf=128, nx=256)
            (act): NewGELUActivation()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (ln_f): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    )
    (lm_head): Linear(in_features=128, out_fea

## ppo前向过程

创建几个model的函数

In [ ]:
def get_response(model, prompt, max_new_tokens, attention_mask):
    # 调用 generate 生成序列；教学示例传入 attention_mask 以避免 pad 干扰
    inputs = {'input_ids': prompt, 'attention_mask': attention_mask}
    y = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        # forced_eos_token_id=True  # 如需强制结束可打开
    )
    return y


def get_reward(model, response, attention_mask):
    # 计算 reward 模型输出；reward_head 仅使用最后一个有效 token
    inputs = {'input_ids': response, 'attention_mask': attention_mask}
    y = model(inputs['input_ids'], inputs['attention_mask'])
    return y


def get_value(model, prompt, attention_mask):
    # 计算 value 序列，每个 token 都输出一个 value，形状 [B, T]
    inputs = {'input_ids': prompt, 'attention_mask': attention_mask}
    y = model(inputs['input_ids'], inputs['attention_mask'])
    return y

In [ ]:
# 查看模拟的 prompt 张量
prompt

tensor([[5, 0, 0, 1, 0],
        [4, 8, 1, 4, 1],
        [9, 6, 7, 0, 5]])

In [ ]:
# 查看模拟的 response 张量
response

tensor([[4, 8, 5, 2, 9, 5, 5, 0, 6, 3],
        [0, 3, 0, 4, 8, 2, 6, 4, 9, 3],
        [2, 6, 7, 5, 0, 0, 3, 3, 4, 8]])

In [ ]:
# 查看 prompt 的 mask（全 1）
prompt_attention_mask

tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])

In [ ]:
# 查看 response 部分的 mask（前半 0，后半 1）
attention_mask

tensor([[0., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 1., 1., 1., 1., 1.]])

在这里就可以看到，ppo流程中的reward只是在最后一个token上得到的，但是我的value model要在每一个token上得到一个价值

In [ ]:
# 依次展示：生成的响应、末 token 奖励、逐 token value（仅示例，不做训练）
print(get_response(model, prompt, max_new_tokens, prompt_attention_mask))
print(get_reward(rm_model, response, attention_mask))
print(get_value(vm_model, response, attention_mask))


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


tensor([[    5,     0,     0,     1,     0,     0,     0,     0,     0,     0],
        [    4,     8,     1,     4,     1, 10998, 10998, 10998, 10998, 10998],
        [    9,     6,     7,     0,     5,     5,     5,     5,     5,     5]])
last_hidden_state shape: torch.Size([3, 128]), sequence_length: torch.Size([3])
tensor([[-0.4702],
        [-1.0223],
        [-0.6396]], grad_fn=<AddmmBackward0>)
tensor([[ 0.1054, -0.1810, -0.2179, -0.4633, -0.1662,  0.0374, -0.7071, -0.7640,
         -1.3427,  0.2779],
        [ 0.0424, -0.0425, -1.1631, -0.1351,  0.2049,  0.0207, -0.9090,  0.4028,
         -0.1427,  0.6911],
        [ 0.1912, -0.2840,  0.1110,  0.6809, -0.4596, -0.1590, -0.2637, -0.3191,
         -0.1446,  0.9440]], grad_fn=<SelectBackward0>)


PPO 相关设置

封装几个ppo的model

In [ ]:
class PPOModels():
    def __init__(self, model_actor, model_ref, model_rm, model_critic):
        # 统一封装 actor/ref/reward/critic，便于传递给训练函数
        self.actor = model_actor
        self.ref = model_ref
        self.rm = model_rm
        self.critic = model_critic

# 参考模型与奖励模型在训练中保持冻结（eval），避免梯度更新
model_ref.eval()
rm_model.eval()
models = PPOModels(model, model_ref, rm_model, vm_model)


设置ppo的超参数

1. ppo_epochs在每次策略更新时，PPO 算法对收集到的数据进行迭代训练的次数。

2. mini_batch_size每个训练步骤中，从收集到的数据里选取的小批量数据的样本数量。

3. epochs整个训练过程中，算法对所有收集到的数据进行完整遍历的次数。

4. kl_ctlKL 散度惩罚项的系数，用于控制新旧策略之间的差异程度。

5. vf_coef价值函数损失的系数，用于平衡策略损失和价值函数损失在总损失中的权重。

6. lam广义优势估计（GAE）中的 \(\lambda\) 参数，用于平衡优势估计的偏差和方差。

7. gamma折扣因子，用于计算未来奖励的折现值，决定未来奖励在当前价值估计中的重要程度。

8. cliprange_value价值函数裁剪范围的参数，用于限制价值函数更新的幅度

In [ ]:
class PPOConfig():
    def __init__(self):
        # ppo_epochs: 每轮 update 中重复遍历批次的次数
        self.ppo_epochs = 5
        # mini_batch_size: 从采样的 batch 中切分出的子批大小
        self.mini_batch_size = 2
        # epochs: 数据集/采样整体遍历轮数（此处未实现完整数据循环，仅示意）
        self.epochs = 4
        # kl_ctl: KL 惩罚系数，约束 actor 偏离 ref 的幅度
        self.kl_ctl = 0.1
        # vf_coef: value loss 权重
        self.vf_coef = 0.1
        # lam: GAE 衰减因子
        self.lam = 0.9
        # gamma: 折扣因子
        self.gamma = 0.9
        # cliprange_value: value 裁剪范围
        self.cliprange_value = 0.2

    def __str__(self):
        return f'ppo_epochs:{self.ppo_epochs}\nmini_batch_size:{self.mini_batch_size}\nepochs:{self.epochs}\nkl_ctl:{self.kl_ctl}'


ppo_config = PPOConfig()

在每一步中ppo都在干什么

首先要有个列表来记录每一步的采样

In [ ]:
# 用字典暂存一次采样得到的各类张量，便于后续计算损失和更新
ppo_old_batchs = {
    'prompt': None,
    'response': None,
    'mask': None,
    'logprobs_ref': None,
    'logprobs_old': None,
    'logprobs': None,
    'values_old': None,
    'values': None,
    'rewards': None,
    'rewards_kl': None,
    'loss': None,
    'logits': None,
}

ppo_old_batchs['prompt'] = prompt
ppo_old_batchs['response'] = response
ppo_old_batchs['mask'] = attention_mask

In [29]:
ppo_old_batchs

{'prompt': tensor([[5, 0, 0, 1, 0],
         [4, 8, 1, 4, 1],
         [9, 6, 7, 0, 5]]),
 'response': tensor([[4, 8, 5, 2, 9, 5, 5, 0, 6, 3],
         [0, 3, 0, 4, 8, 2, 6, 4, 9, 3],
         [2, 6, 7, 5, 0, 0, 3, 3, 4, 8]]),
 'mask': tensor([[0., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 1., 1., 1., 1., 1.]]),
 'logprobs_ref': None,
 'logprobs_old': None,
 'logprobs': None,
 'values_old': None,
 'values': None,
 'rewards': None,
 'rewards_kl': None,
 'loss': None,
 'logits': None}

前向推理，得到token的logprobs

logprobs = F.log_softmax(logits, dim=-1)第一步:对logits进行softmax并取log

torch.gather是一个用于从张量中按索引收集值的操作 

假设我们有:

logp.shape = [1, 5, 32]      # [batch_size, seq_len, vocab_size]

labels.shape = [1, 5]        # [batch_size, seq_len]

1. labels.unsqueeze(2)

在最后增加一个维度

labels_expanded = labels.unsqueeze(2)   # shape变为[1, 5, 1]

2. torch.gather(logp, 2, labels_expanded)

dim=2表示在词表维度(第3维)上收集值

gathered = torch.gather(logp, 2, labels_expanded)  # shape为[1, 5, 1]

3. squeeze(-1)

去掉最后一个维度

logpy = gathered.squeeze(-1)  # 最终shape为[1, 5]

In [ ]:
import torch.nn.functional as F

def get_logits(model, input_ids):
    # 前向得到 logits，形状 [B, T, Vocab]；未传 mask，这里仅演示收集响应 token 的对数概率
    outputs = model(input_ids=input_ids)
    print(f"inputs_ids shape: {input_ids.shape}")
    logits = outputs.logits
    print(f"logits shape: {logits.shape}")
    return logits

def get_logprobs(model, response, attention_mask):
    # 计算指定 response 的逐 token 对数概率
    logits = get_logits(model, response)
    print(f"logits shape: {logits.shape}, response shape: {response.shape}, attention_mask shape: {attention_mask.shape}")
    # 先 softmax 再取对数，得到所有 token 的 logprob
    all_token_logprobs = F.log_softmax(logits, dim=-1)
    print(f"all_token_logprobs shape: {all_token_logprobs.shape}")
    # 根据 response 的 token id 收集对应 logprob；attention_mask 未用于 gather，仅打印形状
    gathered = torch.gather(all_token_logprobs, 2, response.unsqueeze(2))
    print(f"gathered shape: {gathered.shape}, response shape: {response.shape}")
    # 去掉最后一维，得到 [B, T]
    response_logprobs = gathered.squeeze(-1)
    print(f"response_logprobs shape: {response_logprobs.shape}")
    return response_logprobs

# 参考模型 logprob（计算 KL 基线）
logprobs_ref = get_logprobs(models.ref, ppo_old_batchs['response'], ppo_old_batchs['mask'])
print('\n')
# 旧策略（当前 actor）logprob，用于 PPO ratio
logprobs_old = get_logprobs(models.actor, ppo_old_batchs['response'], ppo_old_batchs['mask'])
print('\n')
# 重新计算当前 logprob（示例中与 old 相同，因为未更新参数）
logprobs = get_logprobs(models.actor, ppo_old_batchs['response'], ppo_old_batchs['mask'])

print(logprobs_ref.shape)
print(logprobs_old.shape)
print(logprobs.shape)   


inputs_ids shape: torch.Size([3, 10])
logits shape: torch.Size([3, 10, 50257])
logits shape: torch.Size([3, 10, 50257]), response shape: torch.Size([3, 10]), attention_mask shape: torch.Size([3, 10])
all_token_logprobs shape: torch.Size([3, 10, 50257])
gathered shape: torch.Size([3, 10, 1]), response shape: torch.Size([3, 10])
response_logprobs shape: torch.Size([3, 10])


inputs_ids shape: torch.Size([3, 10])
logits shape: torch.Size([3, 10, 50257])
logits shape: torch.Size([3, 10, 50257]), response shape: torch.Size([3, 10]), attention_mask shape: torch.Size([3, 10])
all_token_logprobs shape: torch.Size([3, 10, 50257])
gathered shape: torch.Size([3, 10, 1]), response shape: torch.Size([3, 10])
response_logprobs shape: torch.Size([3, 10])


inputs_ids shape: torch.Size([3, 10])
logits shape: torch.Size([3, 10, 50257])
logits shape: torch.Size([3, 10, 50257]), response shape: torch.Size([3, 10]), attention_mask shape: torch.Size([3, 10])
all_token_logprobs shape: torch.Size([3, 10, 502

In [32]:
response.shape

torch.Size([3, 10])

In [33]:
logprobs

tensor([[ -9.6364, -10.0382,  -9.4454,  -9.7810,  -9.3484,  -9.5437,  -9.6146,
          -9.3174,  -9.8408,  -9.5032],
        [ -9.6546,  -9.7166,  -9.7343,  -9.4578,  -9.8507,  -9.7604,  -9.8515,
          -9.6053,  -9.3741,  -9.4720],
        [ -9.8447, -10.2057,  -9.4921,  -9.7237,  -9.1873,  -9.4923,  -9.6284,
          -9.9353,  -9.3172,  -9.8445]], grad_fn=<SqueezeBackward1>)

计算kl

In [ ]:
def get_kl(logprobs_ref, logprobs_old, kl_ctl):
    # 逐 token 近似 KL：log p_ref - log p_actor，并乘以系数作为惩罚
    kl = logprobs_ref - logprobs_old
    kl = kl * kl_ctl
    return kl

# 计算 KL 惩罚（未加总），用于 reward 调整
kl = get_kl(logprobs_ref, logprobs_old, ppo_config.kl_ctl)
print(kl)


tensor([[-0.0130,  0.0095, -0.0262, -0.0021, -0.0283, -0.0148, -0.0134, -0.0258,
          0.0089, -0.0307],
        [-0.0315, -0.0049, -0.0047, -0.0323,  0.0020, -0.0178,  0.0170, -0.0316,
         -0.0339, -0.0369],
        [-0.0574,  0.0419, -0.0651, -0.0085, -0.0412, -0.0019, -0.0238,  0.0211,
         -0.0333,  0.0152]], grad_fn=<MulBackward0>)


计算reward_kl


In [ ]:
def get_reward_with_kl(logprobs_ref, logprobs_old, kl_ctl, reward):
    # KL 惩罚加到逐 token 奖励上；末 token 额外叠加 RM 打分，形成 r + KL 约束
    kl = logprobs_ref - logprobs_old
    kl = kl * kl_ctl
    kl[:, -1] += reward[:, 0]
    return kl

print(kl)
# 计算奖励模型输出（末 token），随后与 KL 惩罚合并
rewards = get_reward(models.rm, ppo_old_batchs['response'], ppo_old_batchs['mask'])
print(rewards)

kl_reward = get_reward_with_kl(logprobs_ref, logprobs_old, ppo_config.kl_ctl, rewards)
print(kl_reward)


tensor([[-0.0130,  0.0095, -0.0262, -0.0021, -0.0283, -0.0148, -0.0134, -0.0258,
          0.0089, -0.0307],
        [-0.0315, -0.0049, -0.0047, -0.0323,  0.0020, -0.0178,  0.0170, -0.0316,
         -0.0339, -0.0369],
        [-0.0574,  0.0419, -0.0651, -0.0085, -0.0412, -0.0019, -0.0238,  0.0211,
         -0.0333,  0.0152]], grad_fn=<MulBackward0>)
last_hidden_state shape: torch.Size([3, 128]), sequence_length: torch.Size([3])
tensor([[-0.7784],
        [-0.9515],
        [-0.9003]], grad_fn=<AddmmBackward0>)
tensor([[-0.0130,  0.0095, -0.0262, -0.0021, -0.0283, -0.0148, -0.0134, -0.0258,
          0.0089, -0.8090],
        [-0.0315, -0.0049, -0.0047, -0.0323,  0.0020, -0.0178,  0.0170, -0.0316,
         -0.0339, -0.9884],
        [-0.0574,  0.0419, -0.0651, -0.0085, -0.0412, -0.0019, -0.0238,  0.0211,
         -0.0333, -0.8852]], grad_fn=<CopySlices>)


In [ ]:
# 计算 value 序列（与 actor 共享 backbone 的简化示例）
values = get_value(models.critic, ppo_old_batchs['response'], ppo_old_batchs['mask'])

In [37]:
values

tensor([[ 0.1939, -0.0731, -0.0170, -0.4315,  0.0534, -0.2046, -0.6074, -0.7700,
         -1.2505,  0.1553],
        [ 0.0511, -0.2098, -0.8512, -0.1117,  0.2560, -0.0967, -0.9718,  0.2660,
         -0.1777,  0.4735],
        [ 0.2042, -0.6096, -0.0284,  0.2577, -0.3757, -0.3134, -0.5433, -0.2487,
         -0.2369,  1.0747]], grad_fn=<SelectBackward0>)

In [ ]:
# 将当前计算得到的张量缓存入 batch 字典，便于后续 loss 计算
ppo_old_batchs['logprobs_ref'] = logprobs_ref
ppo_old_batchs['logprobs_old'] = logprobs_old
ppo_old_batchs['logprobs'] = logprobs
ppo_old_batchs['values_old'] = values
ppo_old_batchs['rewards'] = rewards
ppo_old_batchs['rewards_kl'] = kl_reward

ppo_old_batchs

{'prompt': tensor([[5, 0, 0, 1, 0],
         [4, 8, 1, 4, 1],
         [9, 6, 7, 0, 5]]),
 'response': tensor([[4, 8, 5, 2, 9, 5, 5, 0, 6, 3],
         [0, 3, 0, 4, 8, 2, 6, 4, 9, 3],
         [2, 6, 7, 5, 0, 0, 3, 3, 4, 8]]),
 'mask': tensor([[0., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 1., 1., 1., 1., 1.]]),
 'logprobs_ref': tensor([[ -9.7659,  -9.9431,  -9.7075,  -9.8018,  -9.6310,  -9.6916,  -9.7483,
           -9.5755,  -9.7520,  -9.8097],
         [ -9.9691,  -9.7657,  -9.7810,  -9.7806,  -9.8304,  -9.9382,  -9.6816,
           -9.9212,  -9.7132,  -9.8413],
         [-10.4189,  -9.7863, -10.1431,  -9.8084,  -9.5995,  -9.5113,  -9.8666,
           -9.7238,  -9.6501,  -9.6926]], grad_fn=<SqueezeBackward1>),
 'logprobs_old': tensor([[ -9.6364, -10.0382,  -9.4454,  -9.7810,  -9.3484,  -9.5437,  -9.6146,
           -9.3174,  -9.8408,  -9.5032],
         [ -9.6546,  -9.7166,  -9.7343,  -9.4578,  -9.8507,  -9.

## 计算loss

rewards：一个张量，代表在每个时间步获得的奖励。

mask：一个掩码张量，用于标识哪些时间步是有效的（例如，用于处理终止状态）。

values：一个张量，代表每个时间步的状态价值估计。

gamma：折扣因子，用于计算未来奖励的折现值，取值范围通常在 [0, 1] 之间。

lam：GAE 中的 \(\lambda\) 参数，用于平衡偏差和方差，取值范围同样在 [0, 1] 之间。

# PPO 中的 GAE 公式

在PPO（Proximal Policy Optimization）算法中，优势函数和价值损失是连接价值估计与策略优化的核心组件。

## 优势函数（Advantage Function）

优势函数衡量在某一状态下采取特定动作的**相对价值**，定义为：

$$A(s_t, a_t) = Q(s_t, a_t) - V(s_t)$$

状态 - 动作价值函数（Q 函数），表示在状态 \(s_t\) 采取动作 \(a_t\) 后，从后续轨迹中获得的总折扣回报的期望。

状态价值函数（V 函数），表示在状态 \(s_t\) 下，遵循当前策略时获得的总折扣回报的期望（即 “平均收益”）。

优势函数的本质是回答：

在状态 \(s_t\) 下选择动作 \(a_t\)，比‘按当前策略随机选一个动作’好多少？”

若 \(A(s_t, a_t) > 0\)：动作 \(a_t\) 优于平均水平，值得鼓励（策略应提高该动作的概率）

若 \(A(s_t, a_t) < 0\)：动作 \(a_t\) 劣于平均水平，应抑制（策略应降低该动作的概率）。

优势函数将 “绝对价值” 转化为 “相对价值”，减少了估计偏差（例如，即使 \(Q(s_t, a_t)\) 和 \(V(s_t)\) 都有误差，两者的差值可能更稳定）

在实际训练中，Q 和 V 无法直接获得，PPO 通常使用GAE（Generalized Advantage Estimation） 来估计优势函数

GAE（Generalized Advantage Estimation）的时序差分残差公式：

$$\delta_t = r_t + \gamma V(s_{t+1}) - V(s_t)$$

其中，$r_t$ 是时间步 $t$ 的奖励，$\gamma$ 是折扣因子，$V(s_t)$ 是状态 $s_t$ 的价值估计。

GAE 优势估计的递归形式：

$$\hat{A}_t = \delta_t + \gamma \lambda \hat{A}_{t+1}$$

其中 $\lambda$ 是 GAE 的衰减参数（$0 \leq \lambda \leq 1$）。

In [ ]:
def get_GAE(rewards, attention_mask, values, gemma, lam):
    # 基于 GAE 计算优势：先做时间反向累积，再翻转回来；支持掩码对齐有效 token
    lastgae = 0  # 初始化为 0，用于存储上一个时间步的广义优势估计值。
    advantages_recersed = []
    response_len = rewards.shape[-1]

    # 仅在有效 token 上计算，mask 处置零
    values = values * attention_mask
    rewards = rewards * attention_mask

    for t in reversed(range(response_len)):
        nextvalues = values[:, t + 1] if t < response_len - 1 else 0.0
        # TD 误差：r_t + gamma * V_{t+1} - V_t
        delta = rewards[:, t] + gemma * nextvalues - values[:, t]
        # GAE 递推：A_t = delta + gamma * lambda * A_{t+1}
        lastgae = delta + gemma * lam * lastgae
        advantages_recersed.append(lastgae)
    # 反转回正序，得到 [B, T]
    advantages = torch.stack(advantages_recersed[::-1]).transpose(0, 1)
    return advantages


In [40]:
ppo_old_batchs

{'prompt': tensor([[5, 0, 0, 1, 0],
         [4, 8, 1, 4, 1],
         [9, 6, 7, 0, 5]]),
 'response': tensor([[4, 8, 5, 2, 9, 5, 5, 0, 6, 3],
         [0, 3, 0, 4, 8, 2, 6, 4, 9, 3],
         [2, 6, 7, 5, 0, 0, 3, 3, 4, 8]]),
 'mask': tensor([[0., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 1., 1., 1., 1., 1.]]),
 'logprobs_ref': tensor([[ -9.7659,  -9.9431,  -9.7075,  -9.8018,  -9.6310,  -9.6916,  -9.7483,
           -9.5755,  -9.7520,  -9.8097],
         [ -9.9691,  -9.7657,  -9.7810,  -9.7806,  -9.8304,  -9.9382,  -9.6816,
           -9.9212,  -9.7132,  -9.8413],
         [-10.4189,  -9.7863, -10.1431,  -9.8084,  -9.5995,  -9.5113,  -9.8666,
           -9.7238,  -9.6501,  -9.6926]], grad_fn=<SqueezeBackward1>),
 'logprobs_old': tensor([[ -9.6364, -10.0382,  -9.4454,  -9.7810,  -9.3484,  -9.5437,  -9.6146,
           -9.3174,  -9.8408,  -9.5032],
         [ -9.6546,  -9.7166,  -9.7343,  -9.4578,  -9.8507,  -9.

In [ ]:
# 计算优势：使用 KL 调整后的奖励 + 旧 value，得到每个 token 的 A_t
gae = get_GAE(ppo_old_batchs['rewards_kl'], ppo_old_batchs['mask'], ppo_old_batchs['values_old'], ppo_config.gamma, ppo_config.lam)
gae


tensor([[-0.2043, -0.2523, -0.3115, -0.3845, -0.4747, -0.3587, -0.0023,  0.1193,
          0.6180, -0.9643],
        [-0.1865, -0.2303, -0.2843, -0.3509, -0.4333, -0.4275,  0.4546, -0.9550,
         -0.6142, -1.4619],
        [-0.1640, -0.2025, -0.2500, -0.3087, -0.3811, -0.1223,  0.0682, -0.2809,
         -0.4166, -1.9599]], grad_fn=<TransposeBackward0>)

计算value loss


advantages：优势函数的估计值，用于计算回报。


values：当前价值函数的估计值。

values_old：旧的价值函数估计值。

mask：掩码张量，用于指定哪些元素参与损失计算。

cliprange_value：裁剪范围，用于限制价值函数的更新幅度。

https://github.com/huggingface/trl/blob/26d86757a7c7e24e397ea44f57ecce6031dfac01/trl/trainer/ppo_trainer.py#L561C29-L567C30

In [ ]:
def masked_mean(values: torch.Tensor, mask: torch.Tensor, axis = None) -> torch.Tensor:
    """Compute mean of tensor with a masked values."""
    if axis is not None:
        return (values * mask).sum(axis=axis) / mask.sum(axis=axis)
    else:
        return (values * mask).sum() / mask.sum()


def get_value_loss(advantages, values, values_old, attention_mask, cliprange_value):
    # 目标回报 = 旧价值估计 + 优势估计；returns 对齐 PPO 公式中的 V_old + A
    returns = values_old + advantages
    advantages = advantages.detach()
    # 对新的价值估计 values 进行裁剪，限制其与旧价值估计 values_old 的差异不超过 cliprange_value
    vpredclipped = torch.clamp(values, values_old - cliprange_value, values_old + cliprange_value)

    # 两种误差：裁剪版与原版，取较大者以实现价值函数的 PPO-style 裁剪
    vf_losses1 = torch.square(vpredclipped - returns)  # 裁剪后的价值估计与目标回报的平方误差
    vf_losses2 = torch.square(values - returns)        # 未裁剪的价值估计与目标回报的平方误差
    vf_loss_max = torch.max(vf_losses1, vf_losses2)
    vf_loss = 0.5 * masked_mean(vf_loss_max, attention_mask)
    return vf_loss



In [ ]:
# 简化：将新 value 视为旧 value 加常数偏移，便于演示 value loss 裁剪
ppo_old_batchs['values'] = ppo_old_batchs['values_old'] + 0.5

In [ ]:
# 计算 value 损失（带裁剪）
value_loss = get_value_loss(gae, ppo_old_batchs['values'], ppo_old_batchs['values_old'], ppo_old_batchs['mask'], ppo_config.cliprange_value)
value_loss

tensor(0.6554, grad_fn=<MulBackward0>)

计算policy loss
https://github.com/huggingface/trl/blob/26d86757a7c7e24e397ea44f57ecce6031dfac01/trl/trainer/ppo_trainer.py#L569-L574

markdown
# PPO（Proximal Policy Optimization）核心公式与实现

PPO算法的核心是通过策略损失和价值损失的联合优化来更新智能体策略，以下是完整的公式说明与代码实现。

## 1. 策略损失（Policy Loss）

### 核心公式

策略损失的计算基于重要性采样和裁剪机制：

1. **重要性采样比率**  
   $$\text{ratio}_t = \frac{\pi_\theta(a_t | s_t)}{\pi_{\theta_{\text{old}}}(a_t | s_t)} = \exp\left(\log \pi_\theta(a_t | s_t) - \log \pi_{\theta_{\text{old}}}(a_t | s_t)\right)$$

2. **未裁剪损失**  
   $$L_1(\theta) = -A_t \cdot \text{ratio}_t$$

3. **裁剪后损失**  
   $$L_2(\theta) = -A_t \cdot \text{clip}(\text{ratio}_t, 1-\epsilon, 1+\epsilon)$$

4. **最终策略损失**  
   $$L_{\text{policy}}(\theta) = \mathbb{E}\left[ \max(L_1(\theta), L_2(\theta)) \right]$$

其中：
- $A_t$ 是优势估计（GAE计算结果）
- $\epsilon$ 是裁剪范围超参数（通常为0.2）
- $\pi_\theta$ 是当前策略，$\pi_{\theta_{\text{old}}}$ 是更新前的旧策略


In [47]:
def get_policy_loss(advantages, logprobs, logprobs_old, mask, cliprange):
    # 重要性采样
    ratio = torch.exp(logprobs - logprobs_old)
    # 计算策略损失
    pg_losses = -advantages * ratio
    pg_losses2 = -advantages * torch.clamp(ratio, 1.0 - cliprange, 1.0 + cliprange)
    pg_loss_max = torch.max(pg_losses, pg_losses2)
    pg_loss = masked_mean(pg_loss_max, mask)
    return pg_loss



In [ ]:
# 计算策略损失（裁剪版 PPO surrogate）
pg_loss = get_policy_loss(gae, ppo_old_batchs['logprobs'], ppo_old_batchs['logprobs_old'], ppo_old_batchs['mask'], ppo_config.cliprange_value)

In [49]:
pg_loss

tensor(0.4202, grad_fn=<DivBackward0>)

计算熵损失
https://github.com/huggingface/trl/blob/26d86757a7c7e24e397ea44f57ecce6031dfac01/trl/trainer/ppo_trainer.py#L582-L583

entropy（熵）没有直接参与到模型的损失（loss）

在计算完损失并进行反向传播和参数更新后，代码计算了 entropy

这里计算的 entropy 被记录到 entropy_stats 张量中，用于后续的统计和记录，但没有用于损失计算。

In [ ]:
# 生成当前 actor 的 logits，用于后续熵计算
logits = get_logits(models.actor, ppo_old_batchs['response'])
ppo_old_batchs['logits'] = logits

inputs_ids shape: torch.Size([3, 10])
logits shape: torch.Size([3, 10, 50257])


# PPO中的熵损失（Entropy Loss）计算

熵损失用于衡量策略的随机性，在PPO中通常作为总损失的一部分，鼓励智能体保持探索行为。

## 熵计算函数

```python
def get_entropy_loss(logits, mask):
    # 将logits转换为概率分布（softmax归一化）
    prob_dist = torch.nn.functional.softmax(logits, dim=-1)
    
    # 计算熵: H(p) = -Σ(p_i * log(p_i))
    # 等价于: log(Σ(exp(logits_i))) - Σ(p_i * logits_i)
    entropy = torch.logsumexp(logits, dim=-1) - torch.sum(prob_dist * logits, dim=-1)
    
    return entropy

# 计算旧批次数据的熵
entropy = get_entropy_loss(ppo_old_batchs['logits'], ppo_old_batchs['mask'])
entropy  # 返回每个样本的熵值

In [ ]:
def get_entropy_loss(logits, mask):
    # 计算策略熵：H = logsumexp - p*logits，衡量策略的随机性
    prob_dist = torch.nn.functional.softmax(logits, dim=-1)
    print(f"prob_dist shape: {prob_dist.shape}, logits shape: {logits.shape}")
    entropy = torch.logsumexp(logits, dim=-1) - torch.sum(prob_dist * logits, dim=-1)
    return entropy

print(f"logits shape: {logits.shape}, mask shape: {ppo_old_batchs['mask'].shape}")
entropy = get_entropy_loss(ppo_old_batchs['logits'], ppo_old_batchs['mask'])
entropy
                                

logits shape: torch.Size([3, 10, 50257]), mask shape: torch.Size([3, 10])
prob_dist shape: torch.Size([3, 10, 50257]), logits shape: torch.Size([3, 10, 50257])


tensor([[10.7993, 10.7995, 10.7994, 10.7994, 10.7990, 10.7992, 10.7994, 10.7994,
         10.7997, 10.7995],
        [10.7995, 10.7994, 10.7994, 10.7996, 10.7995, 10.7992, 10.7994, 10.7995,
         10.7993, 10.7996],
        [10.7992, 10.7996, 10.7994, 10.7993, 10.7995, 10.7993, 10.7994, 10.7994,
         10.7996, 10.7994]], grad_fn=<SubBackward0>)

In [ ]:
# 总损失：策略损失 + vf_coef * 价值损失（未加入熵项，保持与示例一致）
loss = pg_loss + ppo_config.vf_coef * value_loss

In [ ]:
def get_loss(batchs, ppo_config):
    # 汇总单个小批次的 PPO 损失计算流程
    gae = get_GAE(batchs['rewards_kl'],
                  batchs['mask'],
                  batchs['values'],
                  ppo_config.gamma,
                  ppo_config.lam)
    value_loss = get_value_loss(gae,
                             batchs['values'],
                             batchs['values_old'],
                             batchs['mask'],
                             ppo_config.cliprange_value)
    pg_loss = get_policy_loss(
                              gae,
                              batchs['logprobs'],
                              batchs['logprobs_old'],
                              batchs['mask'],
                              ppo_config.cliprange_value)
    entropy = get_entropy_loss(batchs['logits'], batchs['mask'])  # 仅统计，不计入总损失
    loss = pg_loss + ppo_config.vf_coef * value_loss
    return loss

In [ ]:
# 演示计算一次 batch 的总损失
loss = get_loss(ppo_old_batchs, ppo_config)
loss

prob_dist shape: torch.Size([3, 10, 50257]), logits shape: torch.Size([3, 10, 50257])


tensor(0.9609, grad_fn=<AddBackward0>)

In [56]:
ppo_old_batchs

{'prompt': tensor([[5, 0, 0, 1, 0],
         [4, 8, 1, 4, 1],
         [9, 6, 7, 0, 5]]),
 'response': tensor([[4, 8, 5, 2, 9, 5, 5, 0, 6, 3],
         [0, 3, 0, 4, 8, 2, 6, 4, 9, 3],
         [2, 6, 7, 5, 0, 0, 3, 3, 4, 8]]),
 'mask': tensor([[0., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 1., 1., 1., 1., 1.]]),
 'logprobs_ref': tensor([[ -9.7659,  -9.9431,  -9.7075,  -9.8018,  -9.6310,  -9.6916,  -9.7483,
           -9.5755,  -9.7520,  -9.8097],
         [ -9.9691,  -9.7657,  -9.7810,  -9.7806,  -9.8304,  -9.9382,  -9.6816,
           -9.9212,  -9.7132,  -9.8413],
         [-10.4189,  -9.7863, -10.1431,  -9.8084,  -9.5995,  -9.5113,  -9.8666,
           -9.7238,  -9.6501,  -9.6926]], grad_fn=<SqueezeBackward1>),
 'logprobs_old': tensor([[ -9.6364, -10.0382,  -9.4454,  -9.7810,  -9.3484,  -9.5437,  -9.6146,
           -9.3174,  -9.8408,  -9.5032],
         [ -9.6546,  -9.7166,  -9.7343,  -9.4578,  -9.8507,  -9.

## PPO训练

https://github.com/huggingface/trl/blob/26d86757a7c7e24e397ea44f57ecce6031dfac01/trl/trainer/ppo_trainer.py#L529-L538

将一个完整的批次数据 ppo_batchs 按照指定的 batch_size 和 mini_batch_size 划分成多个小批次数据

In [ ]:
import numpy as np
def get_minibatch(ppo_batchs, batch_size, mini_batch_size):
    # 将完整 batch 打乱并切成多个 mini-batch，用于多次 PPO 更新
    step = batch_size // mini_batch_size
    ppo_batchs_iter = []
    
    # 随机打乱索引以提高训练效果
    b_inds = np.random.permutation(batch_size)
    
    # 根据索引创建小批次
    for i in range(step):
        start_idx = i * mini_batch_size
        end_idx = start_idx + mini_batch_size
        batch_inds = b_inds[start_idx:end_idx]
        
        # 创建当前小批次的数据；仅对 batch 维度匹配的张量做索引
        mini_batch = {}
        for key, value in ppo_batchs.items():
            if value is not None and isinstance(value, torch.Tensor) and value.size(0) == batch_size:
                mini_batch[key] = value[batch_inds]
            else:
                mini_batch[key] = value
                
        ppo_batchs_iter.append(mini_batch)
    
    return ppo_batchs_iter

In [ ]:
# 演示用 Adam 优化器；真实训练需考虑学习率调度与 weight decay 等
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [75]:
ppo_old_batchs

{'prompt': tensor([[5, 0, 0, 1, 0],
         [4, 8, 1, 4, 1],
         [9, 6, 7, 0, 5],
         [4, 8, 5, 2, 9],
         [5, 5, 0, 6, 3]]),
 'response': tensor([[0, 3, 0, 4, 8, 2, 6, 4, 9, 3],
         [2, 6, 7, 5, 0, 0, 3, 3, 4, 8],
         [0, 8, 8, 2, 6, 0, 6, 0, 5, 8],
         [8, 1, 4, 6, 2, 7, 5, 5, 9, 5],
         [7, 4, 9, 5, 6, 6, 6, 1, 9, 8]]),
 'mask': tensor([[0., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 1., 1., 1., 1., 1.]]),
 'logprobs_ref': tensor([[ -9.7657,  -9.5145,  -9.7403,  -9.4521,  -9.8023,  -9.8455,  -9.8040,
           -9.5040,  -9.9263,  -9.4373],
         [-10.0543,  -9.8124,  -9.6533,  -9.7472,  -9.6888,  -9.7347,  -9.5207,
           -9.2883,  -9.4406,  -9.7164],
         [ -9.7657, -10.3167,  -9.8208,  -9.8356,  -9.5770,  -9.7337,  -9.7759,
           -9.6341,  -9.4780,  -9.84

In [ ]:
def ppo_train_step(models, ppo_batchs, ppo_config, get_loss, optimizer):
    # 以当前采样的 batch 为输入，执行多轮（ppo_epochs）PPO 更新；示例未包含重新采样/GAE 更新等完整逻辑
    losses = []
    
    # 多轮PPO训练
    for i in range(ppo_config.ppo_epochs):
        # 获取小批次数据
        ppo_batchs_iter = get_minibatch(
            ppo_batchs, batch_size, ppo_config.mini_batch_size)
        
        # 对每个小批次进行训练
        for mini_batchs in ppo_batchs_iter:
            optimizer.zero_grad()
            # 重新计算所有中间结果，而不是重用之前的计算图
            with torch.set_grad_enabled(True):
                # 这里应重新 forward 计算 logprobs/values/entropy 等；示例仅展示 logits 计算入口
                logits = get_logits(models.actor, mini_batchs['prompt'])
                """
                省略了：根据 mini_batchs 重新计算 logprobs/values/gae/rewards_kl 等，
                以保证梯度针对当前模型参数。
                """

                # 计算损失
                loss = get_loss(
                    mini_batchs, ppo_config)
                
                # 反向传播
                loss.backward()
            optimizer.step()
            
            # 记录损失
            losses.append(loss)
    
    # 更新批次数据中的损失
    ppo_batchs['loss'] = losses
    
    print(losses)

